In [26]:
import requests     # library untuk URL
from bs4 import BeautifulSoup   # library untuk scrapping web
import re
import pandas as pd
from lxml import html
import time
import os
from tqdm import tqdm

In [3]:
data = "full_ptn_politeknik.csv"
df = pd.read_csv(data)
df.head()

,Unnamed: 0,Unnamed: 0.1,NO,KODE,NAMA,JENJANG,DAYA TAMPUNG 2021,PEMINAT 2020,JENIS PORTOFOLIO,KODE_PT,KATEGORI,TIPE
0,0,0,1,1111014,PENDIDIKAN DOKTER HEWAN,S1,94,455.0,Tidak Ada,111,saintek,ptn
1,1,1,2,1111022,TEKNIK SIPIL,S1,77,741.0,Tidak Ada,111,saintek,ptn
2,2,2,3,1111037,TEKNIK MESIN,S1,66,338.0,Tidak Ada,111,saintek,ptn
3,3,3,4,1111045,TEKNIK KIMIA,S1,62,201.0,Tidak Ada,111,saintek,ptn
4,4,4,5,1111053,ARSITEKTUR,S1,61,410.0,Tidak Ada,111,saintek,ptn


Bentuk dari link side panel adalah "https://sidata-ptn.ltmpt.ac.id/ptn_sb.php?ptn=931&prodi=9311016&jenis=0" yang mana terdiri dari link utama"https://sidata-ptn.ltmpt.ac.id/ptn_sb.php" ditambah dengan kode ptn yaitu "?ptn=931" ditambah pula dengan kode prodi yaitu "prodi=9311016" dan diakhiri dengan "&jenis=0".<br>
Berdasarkan analisa link tersebut, untuk mendapatkan seluruh side panel, kode ptn dan kode prodi harus selalu diganti. untuk mendapatkannya cukup menggunakan kolom "KODE" dan "KODE_PT" yang telah dibuat sebelumnya. 

In [15]:
#cek duplicate value kolom kode prodi
df[df.duplicated(['KODE'],keep=False)]

,Unnamed: 0,Unnamed: 0.1,NO,KODE,NAMA,JENJANG,DAYA TAMPUNG 2021,PEMINAT 2020,JENIS PORTOFOLIO,KODE_PT,KATEGORI,TIPE


In [4]:
url = 'https://sidata-ptn.ltmpt.ac.id/ptn_sb.php?ptn='
url1 = '&prodi='
url2 = '&jenis=0'

In [16]:
#membuat link setiap side panel
web=[]
for i in range(len(df)):
    sites = url + str(df['KODE_PT'][i]) + url1 + str(df['KODE'][i]) + url2
    web.append(sites)

In [17]:
web[-5:]

['https://sidata-ptn.ltmpt.ac.id/ptn_sb.php?ptn=832&prodi=8321034&jenis=0',
 'https://sidata-ptn.ltmpt.ac.id/ptn_sb.php?ptn=832&prodi=8321042&jenis=0',
 'https://sidata-ptn.ltmpt.ac.id/ptn_sb.php?ptn=832&prodi=8321057&jenis=0',
 'https://sidata-ptn.ltmpt.ac.id/ptn_sb.php?ptn=832&prodi=8322014&jenis=0',
 'https://sidata-ptn.ltmpt.ac.id/ptn_sb.php?ptn=931&prodi=9311016&jenis=0']

In [19]:
len(web)

3561

In [22]:
#membuat folder baru untuk menyimpan file nanti
os.makedirs("side_panels")

In [27]:
for i in tqdm(range(len(web))):
    url = web[i]
    # membuat get request
    html_content = requests.get(url).text

    # Parse HTML code
    soup = BeautifulSoup(html_content, "lxml")
    
    # mencari semua tabel yang akan discrap
    tabs = soup.find_all("table", attrs={"class":"table table-striped table-condensed"})
    
    #scrape hanya tabel pertama seandainya ada lebih dari satu tabel
    try:
        table = tabs[0]
    except:
        continue
    # menemukan semua table rows
    body = table.find_all("tr")
    # nilai teratas dari table rows merupakan kolom tabel
    head = body[0] # set nama kolom dengan set index body (0)
    body_rows = body[1:] # sisa table rows merupakan isi maka set index body (1:) 

    # iterasi setiap data

    # looping nama kolom
    headings = []
    for item in head.find_all("th"): # nama kolom berada dalam elemen "th"
        # convert elemen "th" berformat text dan strip "\n"
        item = (item.text).rstrip("\n")
        headings.append(item)

    # looping body_rows data

    all_rows = [] #berisi seluruh baris data
    for row_num in range(len(body_rows)):
        row = [] #berisi data per satu baris
        for row_item in body_rows[row_num].find_all("td"): 
            # row_item.text menghapus tags
            # regex menghapus \xa0, \n dan koma dari row_item.text
            aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
            row.append(aa)
        all_rows.append(row)
        
    #menggabungkan kolom dengan body
    df = pd.DataFrame(data=all_rows,columns=headings)
    df['KODE_PT'] = web[i][-25:-22] # set kode pt "https://sidata-ptn.ltmpt.ac.id/ptn_sb.php?ptn=832&prodi=8321034&jenis=0"
    df['KODE_PRODI'] = web[i][-15:-8] # set kode prodi
    #convert DataFrame to .csv
    df.to_csv("side_panels/side_panel_{}.csv".format(web[i][-15:-8]), index=None) # tidak menyimpan index dari dataframe tersebut

100%|██████████████████████████████████████████████████████████████████████████████| 3561/3561 [09:57<00:00,  5.96it/s]
